# Long Condor Spread With Puts

You can think of put condor spread as simultaneously running an in-the-money short put spread and an out-of-the-money long put spread. Ideally, you want the short put spread to expire worthless, while the long put spread achieves its maximum value with strikes C and D in-the-money.

There are four legs to this trade:

* P1 - Buy option whose strike price is three above current stock price
* P2 - Sell option whose strike price is two above current stock price
* P3 - Sell option whose strike price is one above current stock price
* P4 - Buy option whose strike price is one below current strike price

In [1]:
import csv
import numpy as np
import pandas as pd

In [2]:
market_info = pd.read_csv('data-01-market-symbols.csv', index_col='symbol')
market_info = market_info[['previous_close']]
market_info.head(n=1)

,previous_close
symbol,
AACQU,10.47


In [14]:
dfs = []
for x in range(ord('A'), ord('Z')+1):
    dfs.append(pd.read_csv(f'data-02-option_data.{chr(x)}.csv'))
df = pd.concat(dfs)

#
# Limit to one stock for experimentation
#
# df = df[df.symbol.str.startswith('A')]

# Reject call options
df = df[df.type == 'put']

# Reject mark prices less than five cents.
# df = df[df.mark_price > .05]

# Focus on just a few columns
df = df[['symbol', 'expiration_date', 'strike_price', 'mark_price']]

# Add previous close column
df = df.join(market_info, on='symbol')

# Only look at options expiring in the next month.
from datetime import datetime, timedelta
today = datetime.now()
one_month_out = today + timedelta(days=30)
df = df[df.expiration_date < one_month_out.strftime('%Y-%m-%d')]

df['itm'] = df.strike_price < df.previous_close

# Sort so that printing make sense.
df.sort_values(by=['symbol', 'expiration_date', 'strike_price'], ascending=[True, True, False], inplace=True)
df.reset_index(drop=True, inplace=True)

In [15]:
df.head(n=30)

,symbol,expiration_date,strike_price,mark_price,previous_close,itm
0,A,2021-06-18,180.0,42.000,138.13,False
1,A,2021-06-18,155.0,17.400,138.13,False
2,A,2021-06-18,150.0,12.350,138.13,False
3,A,2021-06-18,145.0,7.200,138.13,False
4,A,2021-06-18,140.0,3.550,138.13,False
5,A,2021-06-18,135.0,1.425,138.13,True
6,A,2021-06-18,130.0,0.525,138.13,True
7,A,2021-06-18,125.0,0.225,138.13,True
8,A,2021-06-18,120.0,0.175,138.13,True
9,A,2021-06-18,115.0,0.075,138.13,True


In [16]:
def is_p4(record):
    global p4_puts
    key = f'{record.symbol}/{record.expiration_date}/{record.strike_price}'
    return key in p4_puts

def get_p4_puts(df):
    t2 = df[df.itm == True]
    t3 = t2.groupby(['symbol', 'expiration_date', 'itm']).max('strike_price')
    d = t3.to_dict()
    p4_puts = []
    for x in d['strike_price'].keys():
        symbol = x[0]
        expiration_date = x[1]
        strike_price = d['strike_price'][x]
        p4_puts.append(f'{symbol}/{expiration_date}/{strike_price}')
    return p4_puts

In [17]:
def has_at_least_three_otm_puts(df):
    df = df[df.itm == False]
    return df.shape[0] >= 3

In [ ]:
records = []

t1 = df.groupby(['symbol', 'expiration_date'])
# x is a tuple which has two elements, a tuple (symbol, expiration_date) and a dataframe.
for x in t1:
    symbol = x[0][0]
    expiration_date = x[0][1]
    puts_for_symbol_and_exp = x[1]
    
    #
    # We need at least four puts for this long condor spread.
    #
    if puts_for_symbol_and_exp.shape[0] < 4:
        continue

    #
    # We need at least three OTM puts for this long condor spread.
    #
    if not has_at_least_three_otm_puts(puts_for_symbol_and_exp):
        continue

    p4_puts = get_p4_puts(puts_for_symbol_and_exp)

    t4 = puts_for_symbol_and_exp.copy()
    # This took two minutes
    t4['p1'] = False
    t4['p2'] = False
    t4['p3'] = False
    t4['p4'] = df.apply(is_p4, axis=1)
    t4.reset_index(drop=True, inplace=True)
    
    p4_index = list(t4.columns).index('p4') + 1
    
    lcs = t4.copy()
    for i in lcs.itertuples():
        index = i[0]
        p4 = i[p4_index]
        if p4 == True:
            lcs.at[index-1, 'p3'] = True
            lcs.at[index-2, 'p2'] = True
            lcs.at[index-3, 'p1'] = True
    lcs = lcs[lcs[['p1','p2','p3','p4']].any(1)]
    lcs.drop(['itm'], axis=1, inplace=True)
    lcs.reset_index(drop=True, inplace=True)

    if lcs.shape[0] != 4:
        continue
    
    print(symbol, expiration_date, lcs.shape)
    
    p4 = lcs.iloc[0]
    p3 = lcs.iloc[1]
    p2 = lcs.iloc[2]
    p1 = lcs.iloc[3]

    max_loss = round(p1.mark_price + p4.mark_price - p2.mark_price - p3.mark_price, 2)
    if max_loss == 0:
        max_loss = .05
    if max_loss > .20:
        continue

    max_profit = p3.strike_price - p4.strike_price - max_loss
    ratio = round(max_profit/max_loss, 2)
        
    if ratio < 15:
        print("ration too low")
        continue

    records.append({
        'symbol': symbol,
        'expiration_date': expiration_date,
        'p1': p1.strike_price,
        'p2': p2.strike_price,
        'p3': p3.strike_price,
        'p4': p4.strike_price,
        'max_loss': round(max_loss, 2),
        'max_profit': round(max_profit, 2),
        'ratio': ratio,
    })

df = pd.DataFrame(records)
df.to_csv(f'data-lcs.csv', header=True, index=False, quoting=csv.QUOTE_NONNUMERIC)
df.head()

A 2021-06-18 (4, 9)
AA 2021-06-04 (4, 9)
AA 2021-06-11 (4, 9)
AA 2021-06-18 (4, 9)
AA 2021-06-25 (4, 9)
AACQ 2021-06-18 (4, 9)
AAL 2021-06-04 (4, 9)
AAL 2021-06-11 (4, 9)
AAL 2021-06-18 (4, 9)
AAL 2021-06-25 (4, 9)
AAOI 2021-06-04 (4, 9)
AAOI 2021-06-11 (4, 9)
AAOI 2021-06-18 (4, 9)
AAOI 2021-06-25 (4, 9)
AAON 2021-06-18 (4, 9)
AAP 2021-06-18 (4, 9)
AAPL 2021-06-04 (4, 9)
AAPL 2021-06-11 (4, 9)
AAPL 2021-06-18 (4, 9)
AAPL 2021-06-25 (4, 9)
AAWW 2021-06-18 (4, 9)
ABB 2021-06-18 (4, 9)
ABBV 2021-06-04 (4, 9)
ABBV 2021-06-11 (4, 9)
ABBV 2021-06-18 (4, 9)
ABBV 2021-06-25 (4, 9)
ABC 2021-06-04 (4, 9)
ABC 2021-06-11 (4, 9)
ABC 2021-06-18 (4, 9)
ABC 2021-06-25 (4, 9)
ABCB 2021-06-18 (4, 9)
ABCL 2021-06-18 (4, 9)
ABEV 2021-06-18 (4, 9)
ABG 2021-06-18 (4, 9)
ABM 2021-06-18 (4, 9)
ABMD 2021-06-18 (4, 9)
ABNB 2021-06-04 (4, 9)
ABNB 2021-06-11 (4, 9)
ABNB 2021-06-18 (4, 9)
ABNB 2021-06-25 (4, 9)
ABR 2021-06-18 (4, 9)
ABT 2021-06-04 (4, 9)
ABT 2021-06-11 (4, 9)
ABT 2021-06-18 (4, 9)
ABT 2021-06-25 

ATO 2021-06-18 (4, 9)
ATOM 2021-06-18 (4, 9)
ATOS 2021-06-18 (4, 9)
ATR 2021-06-18 (4, 9)
ATRC 2021-06-18 (4, 9)
ATRO 2021-06-18 (4, 9)
ATSG 2021-06-18 (4, 9)
ATUS 2021-06-18 (4, 9)
ATVI 2021-06-04 (4, 9)
ATVI 2021-06-11 (4, 9)
ATVI 2021-06-18 (4, 9)
ATVI 2021-06-25 (4, 9)
AU 2021-06-18 (4, 9)
AUD 2021-06-18 (4, 9)
AUPH 2021-06-18 (4, 9)
AUY 2021-06-04 (4, 9)
AUY 2021-06-11 (4, 9)
AUY 2021-06-18 (4, 9)
AUY 2021-06-25 (4, 9)
AVAV 2021-06-18 (4, 9)
AVB 2021-06-18 (4, 9)
AVD 2021-06-18 (4, 9)
AVDL 2021-06-18 (4, 9)
AVEO 2021-06-18 (4, 9)
AVGO 2021-06-04 (4, 9)
AVGO 2021-06-11 (4, 9)
AVGO 2021-06-18 (4, 9)
AVGO 2021-06-25 (4, 9)
AVID 2021-06-18 (4, 9)
AVIR 2021-06-18 (4, 9)
AVLR 2021-06-18 (4, 9)
AVNS 2021-06-18 (4, 9)
AVNT 2021-06-18 (4, 9)
AVT 2021-06-18 (4, 9)
AVTR 2021-06-18 (4, 9)
AVXL 2021-06-04 (4, 9)
AVXL 2021-06-11 (4, 9)
AVXL 2021-06-18 (4, 9)
AVXL 2021-06-25 (4, 9)
AVY 2021-06-18 (4, 9)
AVYA 2021-06-18 (4, 9)
AWI 2021-06-18 (4, 9)
AWK 2021-06-18 (4, 9)
AX 2021-06-18 (4, 9)
AXL 2

CDXS 2021-06-18 (4, 9)
CE 2021-06-18 (4, 9)
CECE 2021-06-18 (4, 9)
CELH 2021-06-18 (4, 9)
CENT 2021-06-18 (4, 9)
CENX 2021-06-18 (4, 9)
CERN 2021-06-18 (4, 9)
CERS 2021-06-18 (4, 9)
CEVA 2021-06-18 (4, 9)
CF 2021-06-04 (4, 9)
CF 2021-06-11 (4, 9)
CF 2021-06-18 (4, 9)
CF 2021-06-25 (4, 9)
CFB 2021-06-18 (4, 9)
CFG 2021-06-18 (4, 9)
CFR 2021-06-18 (4, 9)
CFX 2021-06-18 (4, 9)
CG 2021-06-18 (4, 9)
CGBD 2021-06-18 (4, 9)
CGC 2021-06-04 (4, 9)
CGC 2021-06-11 (4, 9)
CGC 2021-06-18 (4, 9)
CGC 2021-06-25 (4, 9)
CGRN 2021-06-18 (4, 9)
CHD 2021-06-18 (4, 9)
CHDN 2021-06-18 (4, 9)
CHEF 2021-06-18 (4, 9)
CHGG 2021-06-18 (4, 9)
CHH 2021-06-18 (4, 9)
CHK 2021-06-18 (4, 9)
CHPT 2021-06-04 (4, 9)
CHPT 2021-06-11 (4, 9)
CHPT 2021-06-18 (4, 9)
CHPT 2021-06-25 (4, 9)
CHRS 2021-06-18 (4, 9)
CHRW 2021-06-18 (4, 9)
CHS 2021-06-18 (4, 9)
CHTR 2021-06-04 (4, 9)
CHTR 2021-06-11 (4, 9)
CHTR 2021-06-18 (4, 9)
CHTR 2021-06-25 (4, 9)
CHUY 2021-06-18 (4, 9)
CHWY 2021-06-04 (4, 9)
CHWY 2021-06-11 (4, 9)
CHWY 2021-06

DB 2021-06-25 (4, 9)
DBD 2021-06-18 (4, 9)
DBI 2021-06-18 (4, 9)
DBX 2021-06-04 (4, 9)
DBX 2021-06-11 (4, 9)
DBX 2021-06-18 (4, 9)
DBX 2021-06-25 (4, 9)
DCI 2021-06-18 (4, 9)
DCOM 2021-06-18 (4, 9)
DCRB 2021-06-18 (4, 9)
DCT 2021-06-18 (4, 9)
DD 2021-06-04 (4, 9)
DD 2021-06-11 (4, 9)
DD 2021-06-18 (4, 9)
DD 2021-06-25 (4, 9)
DDD 2021-06-04 (4, 9)
DDD 2021-06-11 (4, 9)
DDD 2021-06-18 (4, 9)
DDD 2021-06-25 (4, 9)
DDOG 2021-06-04 (4, 9)
DDOG 2021-06-11 (4, 9)
DDOG 2021-06-18 (4, 9)
DDOG 2021-06-25 (4, 9)
DDS 2021-06-18 (4, 9)
DE 2021-06-04 (4, 9)
DE 2021-06-11 (4, 9)
DE 2021-06-18 (4, 9)
DE 2021-06-25 (4, 9)
DECK 2021-06-18 (4, 9)
DELL 2021-06-18 (4, 9)
DEN 2021-06-18 (4, 9)
DENN 2021-06-18 (4, 9)
DEO 2021-06-18 (4, 9)
DFIN 2021-06-18 (4, 9)
DFS 2021-06-04 (4, 9)
DFS 2021-06-11 (4, 9)
DFS 2021-06-18 (4, 9)
DFS 2021-06-25 (4, 9)
DFVL 2021-06-18 (4, 9)
DG 2021-06-04 (4, 9)
DG 2021-06-11 (4, 9)
DG 2021-06-18 (4, 9)
DG 2021-06-25 (4, 9)
DGICA 2021-06-18 (4, 9)
DGLY 2021-06-18 (4, 9)
DGNR 2021

FLR 2021-06-18 (4, 9)
FLR 2021-06-25 (4, 9)
FLUX 2021-06-18 (4, 9)
FLWS 2021-06-18 (4, 9)
FLXN 2021-06-18 (4, 9)
FLY 2021-06-18 (4, 9)
FMAC 2021-06-18 (4, 9)
FMBI 2021-06-18 (4, 9)
FMC 2021-06-18 (4, 9)
FMNB 2021-06-18 (4, 9)
FMTX 2021-06-18 (4, 9)
FMX 2021-06-18 (4, 9)
FN 2021-06-18 (4, 9)
FNB 2021-06-18 (4, 9)
FND 2021-06-18 (4, 9)
FNF 2021-06-18 (4, 9)
FNHC 2021-06-18 (4, 9)
FNLC 2021-06-18 (4, 9)
FNV 2021-06-18 (4, 9)
FOA 2021-06-18 (4, 9)
FOCS 2021-06-18 (4, 9)
FOE 2021-06-18 (4, 9)
FOLD 2021-06-18 (4, 9)
FORM 2021-06-18 (4, 9)
FOSL 2021-06-04 (4, 9)
FOSL 2021-06-11 (4, 9)
FOSL 2021-06-18 (4, 9)
FOSL 2021-06-25 (4, 9)
FOUR 2021-06-18 (4, 9)
FOXA 2021-06-04 (4, 9)
FOXA 2021-06-11 (4, 9)
FOXA 2021-06-18 (4, 9)
FOXA 2021-06-25 (4, 9)
FOXF 2021-06-18 (4, 9)
FPAC 2021-06-18 (4, 9)
FPI 2021-06-18 (4, 9)
FREE 2021-06-18 (4, 9)
FREQ 2021-06-18 (4, 9)
FRG 2021-06-18 (4, 9)
FRGI 2021-06-18 (4, 9)
FRHC 2021-06-18 (4, 9)
FRME 2021-06-18 (4, 9)
FRO 2021-06-18 (4, 9)
FROG 2021-06-18 (4, 9)
FRPT

HPE 2021-06-11 (4, 9)
HPE 2021-06-18 (4, 9)
HPE 2021-06-25 (4, 9)
HPP 2021-06-18 (4, 9)
HPQ 2021-06-04 (4, 9)
HPQ 2021-06-11 (4, 9)
HPQ 2021-06-18 (4, 9)
HPQ 2021-06-25 (4, 9)
HQY 2021-06-18 (4, 9)
HRB 2021-06-18 (4, 9)
HRC 2021-06-18 (4, 9)
HRI 2021-06-18 (4, 9)
HRL 2021-06-04 (4, 9)
HRL 2021-06-11 (4, 9)
HRL 2021-06-18 (4, 9)
HRL 2021-06-25 (4, 9)
HROW 2021-06-18 (4, 9)
HRTG 2021-06-18 (4, 9)
HRTX 2021-06-18 (4, 9)
HRZN 2021-06-18 (4, 9)
HSBC 2021-06-04 (4, 9)
HSBC 2021-06-11 (4, 9)
HSBC 2021-06-18 (4, 9)
HSBC 2021-06-25 (4, 9)
HSC 2021-06-18 (4, 9)
HSKA 2021-06-18 (4, 9)
HST 2021-06-18 (4, 9)
HSY 2021-06-18 (4, 9)
HSY 2021-06-25 (4, 9)
HT 2021-06-18 (4, 9)
HTBK 2021-06-18 (4, 9)
HTLD 2021-06-18 (4, 9)
HTOO 2021-06-18 (4, 9)
HUBB 2021-06-18 (4, 9)
HUBG 2021-06-18 (4, 9)
HUBS 2021-06-18 (4, 9)
HUGE 2021-06-18 (4, 9)
HUM 2021-06-04 (4, 9)
HUM 2021-06-11 (4, 9)
HUM 2021-06-18 (4, 9)
HUM 2021-06-25 (4, 9)
HUN 2021-06-18 (4, 9)
HURN 2021-06-18 (4, 9)
HUYA 2021-06-04 (4, 9)
HUYA 2021-06-11

LBRT 2021-06-18 (4, 9)
LBTYA 2021-06-18 (4, 9)
LBTYK 2021-06-04 (4, 9)
LBTYK 2021-06-11 (4, 9)
LBTYK 2021-06-18 (4, 9)
LBTYK 2021-06-25 (4, 9)
LC 2021-06-18 (4, 9)
LCI 2021-06-18 (4, 9)
LCII 2021-06-18 (4, 9)
